In [1]:
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#读取数据
data = pd.read_excel('./预测人口.xls')
# 先训练总人口
train = data['年末总人口(万人)'].values[:-15] #训练数据
test = data['年末总人口(万人)'].values[-15:] #测试数据
data

,时间,城镇人口(万人),乡村人口(万人),出生人口,中国性别比例(按照女生=100),总和生育率,自然增长率,结婚率,国民总收入(亿元),年末总人口(万人)
0,1980,19140,79565,1776,105.98,2.61,11.87,14.6,4586.1,98705
1,1981,20171,79901,2064,106.11,2.55,14.55,20.8,4933.7,100072
2,1982,21480,80174,2230,106.19,2.54,15.68,16.5,5380.5,101654
3,1983,22274,80734,2052,106.61,2.56,13.29,14.9,6043.8,103008
4,1984,24017,80340,2050,106.61,2.61,13.08,15.0,7314.2,104357
5,1985,25094,80757,2196,107.04,2.65,14.26,15.7,9123.6,105851
6,1986,26366,81141,2374,107.04,2.67,15.57,16.4,10375.4,107507
7,1987,27674,81626,2508,106.19,2.64,16.61,17.2,12166.6,109300
8,1988,28661,82365,2445,106.27,2.58,15.73,16.6,15174.4,111026
9,1989,29540,83164,2396,106.40,2.46,15.04,16.8,17188.4,112704


In [2]:
len(test)

15

In [3]:
def GM11(x,n):
    '''
    灰色预测
    x：序列，numpy对象
    n:需要往后预测的个数
    '''
    x1 = x.cumsum()#一次累加  
    z1 = (x1[:len(x1) - 1] + x1[1:])/2.0#紧邻均值  
    z1 = z1.reshape((len(z1),1))  
    B = np.append(-z1,np.ones_like(z1),axis=1)  
    Y = x[1:].reshape((len(x) - 1,1))
    
    #a为发展系数 b为灰色作用量
    [[a],[b]] = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)#计算参数  
    result = (x[0]-b/a)*np.exp(-a*(n-1))-(x[0]-b/a)*np.exp(-a*(n-2))  
    S1_2 = x.var()#原序列方差
    e = list()#残差序列
    for index in range(1,x.shape[0]+1):
        predict = (x[0]-b/a)*np.exp(-a*(index-1))-(x[0]-b/a)*np.exp(-a*(index-2))
        e.append(x[index-1]-predict)
    S2_2 = np.array(e).var()#残差方差
    C = S2_2/S1_2#后验差比
    if C<=0.35:
        assess = '后验差比<=0.35，模型精度等级为好'
    elif C<=0.5:
        assess = '后验差比<=0.5，模型精度等级为合格'
    elif C<=0.65:
        assess = '后验差比<=0.65，模型精度等级为勉强'
    else:
        assess = '后验差比>0.65，模型精度等级为不合格'
        
    #预测数据
    predict = list()
    for index in range(x.shape[0]+1,x.shape[0]+n+1):
        predict.append((x[0]-b/a)*np.exp(-a*(index-1))-(x[0]-b/a)*np.exp(-a*(index-2)))
    predict = np.array(predict)
    return {
            'a':{'value':a,'desc':'发展系数'},
            'b':{'value':b,'desc':'灰色作用量'},
            'predict':{'value':result,'desc':'第%d个预测值'%n},
            'C':{'value':C,'desc':assess},
            'predict':{'value':predict,'desc':'往后预测%d个的序列'%(n)},
            }

In [4]:
#GM11动态建模
yPre = []
for i in range(test.shape[0]):
    #只预测1个数
    result = GM11(train,1)
    yPre.append(result['predict']['value'][0])
    #更新训练集
    train = train.tolist()[:-1]
    train.append(test[i])
    train = np.array(train).reshape(-1)
#计算MAE
MAE = mean_absolute_error(test,yPre)
#打印模型
print(result['C']['desc'])
print(result['a']['desc'],np.round(result['a']['value'],2))
print(result['b']['desc'],np.round(result['b']['value'],2))

后验差比<=0.35，模型精度等级为好
发展系数 -0.01
灰色作用量 99743.64


In [5]:
# i = 21
# for i in range(2,41):
#     data = np.array([98705,100072,101654,103008,104357,105851,107507,109300,111026,112704,114333,115823,117171,118517,119850,121121,122389
#     ,123626,124761,125786,126743,127627,128453,129227,129988,130756,131448,132129,132802,133450,134091,134916,135922,136726,137646,138326
#     ,139232,140011,140541,141008,141212])
#     x =  data[:i] #输入数据 1994年之前
#     y =  data[i:i+1] #需要预测的数据 2001-2020
#     result = GM11(x,len(y))  
#     predict = result['predict']['value']
#     predict = np.round(predict,1)
#     print('第',1980+i,'的真实值:',y)
#     print('预测值:',predict)
#     print()

In [6]:
data['年末总人口(万人)']

0      98705
1     100072
2     101654
3     103008
4     104357
5     105851
6     107507
7     109300
8     111026
9     112704
10    114333
11    115823
12    117171
13    118517
14    119850
15    121121
16    122389
17    123626
18    124761
19    125786
20    126743
21    127627
22    128453
23    129227
24    129988
25    130756
26    131448
27    132129
28    132802
29    133450
30    134091
31    134916
32    135922
33    136726
34    137646
35    138326
36    139232
37    140011
38    140541
39    141008
40    141212
Name: 年末总人口(万人), dtype: int64

In [7]:
if __name__ == "__main__":
    data = np.array(data['年末总人口(万人)'])
    x = data[:26] #输入数据
    y = data[26:] #需要预测的数据
    result = GM11(x,len(y))  
    predict = result['predict']['value']
    predict = np.round(predict,1)
    print('真实值:',y)
    print('预测值:',predict)

真实值: [131448 132129 132802 133450 134091 134916 135922 136726 137646 138326
 139232 140011 140541 141008 141212]
预测值: [135137.8 136653.7 138186.6 139736.8 141304.3 142889.3 144492.2 146113.
 147752.1 149409.5 151085.5 152780.3 154494.1 156227.1 157979.6]


In [8]:
len(y)

15

In [9]:
print(result) # 2021-2035的相关数据

{'a': {'value': -0.011155054993812277, 'desc': '发展系数'}, 'b': {'value': 100579.4959180456, 'desc': '灰色作用量'}, 'predict': {'value': array([135137.82430287, 136653.73347043, 138186.64735608, 139736.75671031,
       141304.25442336, 142889.33554925, 144492.19732998, 146113.03922013,
       147752.06291166, 149409.47235901, 151085.47380448, 152780.2758039 ,
       154494.08925257, 156227.1274115 , 157979.60593398]), 'desc': '往后预测15个的序列'}, 'C': {'value': 0.02152912749023974, 'desc': '后验差比<=0.35，模型精度等级为好'}}


In [10]:
print(list(result['predict']['value']))
print(len(list(result['predict']['value'])))

[135137.8243028745, 136653.73347043432, 138186.64735607803, 139736.7567103058, 141304.25442336313, 142889.33554925397, 144492.19732997753, 146113.0392201282, 147752.06291165762, 149409.47235900722, 151085.4738044832, 152780.27580389753, 154494.08925256692, 156227.12741149776, 157979.6059339773]
15


In [11]:
len(list(result['predict']['value']))

15